In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [8]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

In [9]:
rf = RandomForestClassifier(n_jobs=-1,
                            # 모델의 학습과 예측에서 사용할 cpu 코어의 수를 지정
                            # 이를 통해 모델을 학습, 예측을 수행
                            random_state=42)

In [10]:
scores = cross_validate(
    rf, train_input, train_target,
    return_train_score=True, n_jobs=-1)

In [11]:
# return_train_score 검증 점수 뿐만 아니라 

In [12]:
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 훈련에서의 성능이 테스트에서의 성능보다 현저히 높으면 일반적으로 과적합

0.9973541965122431 0.8905151032797809


In [13]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [15]:
rf = RandomForestClassifier(oob_score=True,
                            # out of bag 샘플에 포함되지 않은 데이터
                            n_jobs=-1, 
                            # 병렬 처리를 위한 cpu 코어 수를 설정
                            random_state=42)
                            # 실험의 재현성을 보장한다

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


# 엑스트라 트리

In [16]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


# 그레디언트 부스팅

In [17]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(
    gb, train_input, train_target,
    return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [18]:
gb = GradientBoostingClassifier(
    n_estimators=500,
    learning_rate=0.2,
    random_state=42)
scores = cross_validate(
    gb, train_input, train_target,
    return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [19]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15853457 0.68010884 0.1613566 ]


# 히스토그램 기반 부스팅

In [20]:

# 사이킷런 1.0 버전 아래에서는 다음 라인의 주석을 해제하고 실행하세요.
# from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [21]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [22]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [23]:
hgb.score(test_input, test_target)

0.8723076923076923

# XGBoost

In [24]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9558403027491312 0.8782000074035686


# LightGBM

In [26]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
